In [1]:
# ABM
import mesa

# Data viz
import plotly_express as px

# data analysis
import numpy as np
import pandas as pd

### Agent

In [140]:
#import copy

class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        # Pass the parameters to the parent class.
        super().__init__(unique_id, model)

        # Create the agent's variable and set the initial values.
        self.wealth = 1

    def step(self):
        # Verify agent has some wealth
        if self.wealth > 0:
            if self.model.network:
                connected_agents = self.get_connected_agents()
                if connected_agents:
                    recipient_id = self.random.choice(connected_agents)
                    other_agent = self.model.schedule.agents[recipient_id]
                else:
                    other_agent = None
            else:
                other_agent = self.random.choice(self.model.schedule.agents)
            if other_agent is not None:
                other_agent.wealth += 1
                self.wealth -= 1
    
    def get_connected_agents(self):
        # Get indices of connected agents from the adjacency matrix
        connected = np.nonzero(self.model.adjacency_matrix[self.unique_id])[0]
        return list(connected)

### Model

In [74]:
class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, M, ps=0.5, network=True):
        super().__init__()
        self.num_agents = N
        self.M = M
        self.network = network
        # Create scheduler and assign it to the model
        self.schedule = mesa.time.RandomActivation(self)
        # adjacency matrix
        self.adjacency_matrix = np.zeros((N, N), dtype=int)
        # preferential attachment strength
        self.preference_strength = ps

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            # Add the agent to the scheduler
            self.schedule.add(a)
        
        # initialize agent network
        if self.network:
            self.init_network()
        
    def init_network(self):
        for i in range(1, self.num_agents):
            degrees = self.adjacency_matrix.sum(axis=0).astype(float)  # Degree of each agent
            if self.preference_strength >= 0:
                # Attraction: higher degree increases the chance of new connections
                probabilities = (degrees + 1) ** self.preference_strength
            else:
                # For ps < 0, calculate probabilities for the mirrored ps value and then flip the probabilities about the mean degree
                mirrored_ps = abs(self.preference_strength)
                mirrored_probabilities = degrees ** mirrored_ps
                # Flip probabilities about the mean degree
                probabilities = np.array([mirrored_probabilities[-(j+1)] for j in range(len(degrees))])

            # don't choose self, i
            probabilities[i] = 0
            # Normalize to get probabilities
            probabilities /= probabilities.sum()

            for _ in range(self.M):
                # Choose an agent to connect with based on probabilities
                chosen_agent = np.random.choice(self.num_agents, p=probabilities)
                # Create a bi-directional connection
                self.adjacency_matrix[i, chosen_agent] = 1
                self.adjacency_matrix[chosen_agent, i] = 1
                
    def step(self):
        """Advance the model by one step."""

        # The model's step will go here for now this will call the step method of each agent and print the agent's unique_id
        self.schedule.step()

### Run it

In [130]:
model = MoneyModel(10, 5)
for i in range(10):
    model.step()

In [82]:
np.sum(np.tril(model.adjacency_matrix, -1))
model.adjacency_matrix

array([[0, 1, 0, 1, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 0, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 1, 1, 0, 1, 0]])

### Extract model data

In [83]:
agent_wealth = [a.wealth for a in model.schedule.agents]
agent_wealth

[1, 1, 2, 1, 0, 0, 0, 2, 2, 1]

### Plot

In [80]:
# Create a histogram with plotly_express
fig = px.histogram(
    agent_wealth, 
    nbins=len(set(agent_wealth)), # This ensures discrete bins if needed
    labels={'value': 'Wealth'}, # Renaming the default 'value' label to 'Wealth'
    title="Wealth Distribution"
)
fig.update_layout(
    xaxis_title="Wealth",
    yaxis_title="Number of Agents"
)
fig.show()

### Gini

In [202]:
def gini_coefficient(wealths):
    """Calculate the Gini coefficient from a list of wealths."""
    n = len(wealths)
    if n == 0:
        return None  # Avoid division by zero if wealths is empty
    
    # Ensure the wealth list is sorted.
    sorted_wealths = sorted(wealths)
    total_wealth = sum(sorted_wealths)
    
    # Check for total wealth to avoid division by zero in completely equal societies.
    if total_wealth == 0:
        return 0
    
    cumulative_wealth_sum = 0
    weighted_sum = 0
    for i, wealth in enumerate(sorted_wealths, 1):
        cumulative_wealth_sum += wealth
        weighted_sum += i * wealth
    
    # The Gini coefficient is half the relative mean absolute difference.
    gini = (2 * weighted_sum) / (n * total_wealth) - (n + 1) / n
    return gini

### Multiple runs

In [213]:
all_wealth = []
gini = []
# This runs the model 100 times, each model executing 10 steps.
for j in range(100):
    run_wealth = []
    # Run the model
    model = MoneyModel(10, 10, 1, True)
    for i in range(10):
        model.step()

    # Store the results
    for agent in model.schedule.agents:
        all_wealth.append(agent.wealth)
        run_wealth.append(agent.wealth)
    
    # run wealth
    gini.append(gini_coefficient(run_wealth))

In [214]:
run_wealth
gini

[0.45999999999999996,
 0.5399999999999998,
 0.31999999999999984,
 0.6399999999999999,
 0.45999999999999996,
 0.6599999999999999,
 0.48,
 0.33999999999999986,
 0.74,
 0.6199999999999999,
 0.7,
 0.45999999999999996,
 0.45999999999999996,
 0.5799999999999998,
 0.45999999999999996,
 0.5799999999999998,
 0.5399999999999998,
 0.72,
 0.5599999999999998,
 0.5799999999999998,
 0.76,
 0.6399999999999999,
 0.5799999999999998,
 0.5399999999999998,
 0.5399999999999998,
 0.5999999999999999,
 0.6599999999999999,
 0.6399999999999999,
 0.41999999999999993,
 0.5399999999999998,
 0.41999999999999993,
 0.76,
 0.5399999999999998,
 0.5799999999999998,
 0.41999999999999993,
 0.6399999999999999,
 0.6799999999999999,
 0.45999999999999996,
 0.5399999999999998,
 0.6199999999999999,
 0.7,
 0.8199999999999998,
 0.5399999999999998,
 0.45999999999999996,
 0.72,
 0.5599999999999998,
 0.5399999999999998,
 0.5599999999999998,
 0.48,
 0.76,
 0.72,
 0.5399999999999998,
 0.5599999999999998,
 0.45999999999999996,
 0.319999

In [215]:
fig = px.histogram(
    all_wealth, 
    nbins=len(set(all_wealth)), # This ensures discrete bins if needed
    labels={'value': 'Wealth'}, # Renaming the default 'value' label to 'Wealth'
    title="Wealth Distribution"
)
fig.update_layout(
    xaxis_title="Wealth",
    yaxis_title="Number of Agents"
)
fig.show()

### Preferential Attachment

In [216]:
# Define degrees range
degrees = np.arange(1.0, 11.0)  # From 1 to 10
mean_degree = np.mean(degrees)

# Define preference_strength values for examples
preference_strengths = [-1, -0.5, 0, 0.5, 1]

# Prepare data for plotting
data = []
for ps in preference_strengths:
    if ps > 0:
        probabilities = degrees ** ps
    else:
        # For ps < 0, calculate probabilities for the mirrored ps value and then flip the probabilities about the mean degree
        mirrored_ps = abs(ps)
        mirrored_probabilities = degrees ** mirrored_ps
        # Flip probabilities about the mean degree
        probabilities = np.array([mirrored_probabilities[-(i+1)] for i in range(len(degrees))])
    
    probabilities /= probabilities.sum()  # Normalize
    for degree, probability in zip(degrees, probabilities):
        data.append({'Degree': degree, 'Probability': probability, 'Preference Strength': f'ps={ps}'})

df = pd.DataFrame(data)

# Plot using Plotly Express
fig = px.line(df, x='Degree', y='Probability', color='Preference Strength',
              title='Adjusted Effect of Preference Strength on Connection Probability',
              labels={'Probability': 'Normalized Probability', 'Degree': 'Degree'})

# Customize plot
fig.update_layout(legend_title_text='Preference Strength')
fig.show()